# Tomas Murphy 06488587

## Imports

In [8]:
from DataStructures import create_airport_list, createAircrafts, getRoute, Airport
import pandas as pd
test_df = pd.read_csv("test.csv")
airport_df = pd.read_csv("airport.csv")
country_currency_df = pd.read_csv("countrycurrency.csv")
currency_rates_df = pd.read_csv("currencyrates.csv")
aircraft_df = pd.read_csv("aircraft.csv")
airport_currencies_df = airport_df.merge(country_currency_df.rename(columns={"name":"Country"}), on="Country")
merged_df = airport_currencies_df.merge(currency_rates_df.rename(columns={"CurrencyCode":"currency_alphabetic_code"}), on="currency_alphabetic_code")
airports = create_airport_list(merged_df)
aircrafts = createAircrafts(aircraft_df)

In [9]:
# Import pandas and math 
import pandas as pd
import math
import unittest

# My Aircraft Class

#### This class was made to create aircraft for the assignment. A full list of methods and their descriptions can be found in the comments in the cell below. 

#### Correctness: Passed all unit tests

#### Speed: 5.46 ms to read the aircraft data from a CSV, create aircraft objects based on the data, store them in a list, and print them. Due to using basic python code, there are no performance issues.

#### Efficiency: Setting and getting variables in a standard Python way. Constant time operations. 

#### Security: Name mangling for "private" variable types. Getters and setters (encapsulation).

#### Robustness: The class will run efficiently if the passed CSV files contain the correct data. This inputted is tested and altered as necessary (see design).

#### Clarity: Code is indented and commented, with multiple cells used to break up code.

#### Maintainability: Code commented and laid out in segments for easier searching and alterations. Encapsulation helps with changing functionality behind the scenes.

### Design

#### Scalabilty: This class could be reused in another program which takes fuel capacity and units into account. As many aircraft as needed can be made with the class.

#### Robustness: Conversion from imperial to metric to ensure kilometers are used. If/else used to ensure plane isn't overfuelled. 

### Note: This was how the class looked originally. Later I realised that an aircraft's max fuel equals the max distance as 1L = 1KM, so if two airports fall within an aircraft's max distance, then we just had to multiply the distance by the currency to get the cost. Therefore, only the __init__ and get_maxfuel methods were used in the final code; the rest was superfluous.

In [10]:

# Create class for aircraft

class Aircraft:
    
    # Methods included in the class:
    """Methods
    __init__:               Object initialisation
    __repr__:               Print out Aircraft information
    set_fuel:               Set the fuel amount
    get_fuel:               Return the fuel amount
    get_name:               Return the Aircraft name
    get_units:              Return the Aircraft units
    get_maxfuel:            Return the Aircraft's max fuel
    refuel:                 Return confirmation if plane refueled
    get_fuel_needed:        Return the fuel needed for a route
    find_journey_cost:      Return the cost for a route
    """

    def __init__(self, name='', units='metric', fuelcapacity=0 ):
        self.__name = name    # Aircraft name
        self.__fuel = 0       # Current fuel in aircraft
        self.__max_Fuel = fuelcapacity # Max fuel capacity (range is 1km per 1 litre, so aircraft range = max fuel capacity)
        self.__units = units # units of measurement: metric or imperial
        
        
        # If the range is not in metric, convert imperial to metric
        if self.__units != "metric":
            self.__max_Fuel = round((self.__max_Fuel * 1.60934),2)
            
            
        # Used to print out details of the aircraft (used for testing purposes mainly)
    def __repr__(self):
        return "Aircraft name: %s aircraft fuel capacity: %s Aircraft units: %s" % (self.get_name(), self.get_maxfuel(), 
                                                                                    self.get_units())
      
       # Set the fuel level of the aircraft
    def set_fuel(self, fuel):
        if self.__fuel + fuel <= self.__max_Fuel:    # Ensure the fueling doesn't exceed the aircraft's max capacity
            self.__fuel = fuel
        else:
            print("Improper amount requested.")  
            
        # Get the current fuel levels
    def get_fuel(self):
        return self.__fuel

        # Get the aircraft name
    def get_name(self):
        return self.__name
    
        # Get the unit type: metric or imperial
    def get_units(self):
        return self.__units
    
        # Get fuel capacity
    def get_maxfuel(self):
        return self.__max_Fuel
      
      # Add necessary amount of fuel for the flight
    def refuel(self, current_airport, destination_airport): 
        fuel_needed = self.get_fuel_needed(current_airport, destination_airport)
        self.set_fuel(fuel_needed)
        return "Plane refueled"

      # Calculate fuel needed for the flight
    def get_fuel_needed(self, current_airport, destination):
        fuel_needed = current_airport.get_distance(destination)
        return fuel_needed
    
# Constant time operations
      
     


## Create a dictionary containing every Aircraft object

In [11]:
def createAircrafts(aircraft_df):
    aircrafts = {}
    for i in range(len(aircraft_df)):
        name = aircraft_df["code"][i]
        units = aircraft_df["units"][i]
        fuelcap = aircraft_df["range"][i]
        aircrafts[name] = Aircraft(name, units, fuelcap)
    return aircrafts

# n time complexity (linear)

In [12]:
%%time
createAircrafts(aircraft_df)

Wall time: 995 µs


{'A319': Aircraft name: A319 aircraft fuel capacity: 3750 Aircraft units: metric,
 'A320': Aircraft name: A320 aircraft fuel capacity: 12000 Aircraft units: metric,
 'A321': Aircraft name: A321 aircraft fuel capacity: 12000 Aircraft units: metric,
 'A330': Aircraft name: A330 aircraft fuel capacity: 13430 Aircraft units: metric,
 '737': Aircraft name: 737 aircraft fuel capacity: 9012.3 Aircraft units: imperial,
 '747': Aircraft name: 747 aircraft fuel capacity: 15771.53 Aircraft units: imperial,
 '757': Aircraft name: 757 aircraft fuel capacity: 11622.65 Aircraft units: imperial,
 '767': Aircraft name: 767 aircraft fuel capacity: 11474.59 Aircraft units: imperial,
 '777': Aircraft name: 777 aircraft fuel capacity: 15610.6 Aircraft units: imperial,
 'BAE146': Aircraft name: BAE146 aircraft fuel capacity: 2909 Aircraft units: metric,
 'DC8': Aircraft name: DC8 aircraft fuel capacity: 7724.83 Aircraft units: imperial,
 'F50': Aircraft name: F50 aircraft fuel capacity: 2055 Aircraft units:

## Unit tests

In [13]:
%%time

class aircraft_unit_tests(unittest.TestCase):
    
    # An aircraft will have a default fuel of 0 after creation.
    
        def test_get_fuel(self):
            a = Aircraft("Boeing101", "metric", 500)
            self.assertTrue(a.get_fuel() == 0)

        def test_get_name(self):
            a = Aircraft("Boeing101", "metric", 500)
            self.assertTrue(a.get_name() == "Boeing101")

        def test_repr(self):
            a = Aircraft("Boeing101", "metric", 1000)
            self.assertTrue(a.__repr__() == "Aircraft name: Boeing101 aircraft fuel capacity: 1000 Aircraft units: metric")

        def test_get_distance(self):
            lhr = Airport("LHR", 51.470600, -0.461941, 1.4029)
            dub = Airport("DUB", 53.421299, -6.270070)
            self.assertTrue(lhr.get_distance(dub) == 449)
            
        def test_refuel(self):
            lhr = Airport("LHR", 51.470600, -0.461941, 1.4029)
            dub = Airport("DUB", 53.421299, -6.270070)
            a = Aircraft("Boeing101", "metric", 500)
            a.refuel(lhr, dub)
            self.assertTrue(a.get_fuel() == 449)

        def test_get_maxfuel(self):
            a = Aircraft("Boeing101", "metric", 500)
            self.assertTrue(a.get_maxfuel() == 500)
            
        def test_get_units(self):
            a = Aircraft("Boeing101", "imperial", 500)
            self.assertTrue(a.get_units() == "imperial")

            
        def test_get_fuel_needed(self):
            lhr = Airport("LHR", 51.470600, -0.461941, 1.4029)
            dub = Airport("DUB", 53.421299, -6.270070)
            a = Aircraft("Boeing101", "metric", 500)
            self.assertTrue(a.get_fuel_needed(lhr,dub) == 449)
            
        def test_create_aircraft(self):
            createAircrafts(aircraft_df)
            self.assertTrue(len(aircrafts) == 20)
        
    
unittest.main(argv=[''], verbosity=2, exit=False)


test_create_aircraft (__main__.aircraft_unit_tests) ... ok
test_get_distance (__main__.aircraft_unit_tests) ... ok
test_get_fuel (__main__.aircraft_unit_tests) ... ok
test_get_fuel_needed (__main__.aircraft_unit_tests) ... ok
test_get_maxfuel (__main__.aircraft_unit_tests) ... ok
test_get_name (__main__.aircraft_unit_tests) ... ok
test_get_units (__main__.aircraft_unit_tests) ... ok
test_refuel (__main__.aircraft_unit_tests) ... ok
test_repr (__main__.aircraft_unit_tests) ... 

Wall time: 11 ms


ok

----------------------------------------------------------------------
Ran 9 tests in 0.009s

OK


# Brute force route finding (unoptimised)

#### These methods were created to find the cheapest route by creating all permutations, choosing only those which started with the start airport chosen, and then calculating the cost of these remaining routes. 

#### Correctness: Passed all unit tests

#### Speed: 

Create Permutations: 8.58 µs

Remove unwanted permutations: 6.6 µs

Calculate cost per permutation: 926 µs

#### Efficiency: 
Create Permutations: n!

Remove unwanted permutations: n

Calculate cost per permutation: n^2

#### Security: The code is in methods, and thus aids encapsulation.

#### Robustness: If jumps are invalid or impossible, the next route is considered.

#### Clarity: Code is indented and commented, with multiple cells used to break up code.

#### Maintainability: Code commented and laid out in segments for easier searching and alterations. Encapsulation helps with changing functionality behind the scenes.

### Design

#### Scalabilty: These methods can handle any number of input, but due to the n! big O of finding permutations, it may become slow for very large inputs.

#### Robustness: If an airport is out of range, then that route is disregarded and the next route is checked, and so on.

In [14]:
import itertools
# Get all route permutations 
def createPermutations(route):
    return list(itertools.permutations(route))

# n!

In [15]:
# Get permutations where the start airport = the inputted airport, and end airport is = "HOM".

def removeUnwantedPermutations(airport, permutations):
    desiredPermutations = []
    for i in permutations:
        if i[0] == airport:
            desiredPermutations.append(i)
    return desiredPermutations

# n runtime complexity

In [16]:
# Calculate the costs of the remaining permutations and return the route with the lowest cost. 

def calculateCostPerPermutation(permutations, aircraftRange, airports):
    # Store routes and associated costs in a dictionary
    cost_dict = {}
    # The start is the first airport in the permutations
    start = permutations[0][0]
    ans = ""
    # Loop through the permutations
    for i in permutations:
        route = [start]
        total = 0
        # Loop through the airports in the permutation
        for j in range(1, len(i)):
            # Get the distance between the airports in the permutation, if it's out of range, break
            nextJump = airports.search(i[j-1]).get_distance(airports.search(i[j]))
            if nextJump > aircraftRange:
                break
                # Multiply distance by cost, and add the cost to the total
            total += nextJump * airports.search(i[j-1]).get_conversion_rate()
                # Add the airports to the route
            route.append(i[j])
            
            
        # Calculate jump to return home
        nextJump = airports.search(i[-1]).get_distance(airports.search(start))
        # If the jump to the start airport is not possible, we continue to the next permutation
        if nextJump > aircraftRange:
            continue
        # Calculate trip cost to the start airport
        total += nextJump * airports.search(i[-1]).get_conversion_rate()
        # Add the trip to the start airport to the route
        route += [start]    
        # Store the cost and route in the dictionary
        cost_dict[total] = route
    # Find the minimum cost
    minimum_cost = min(cost_dict)
    # Find the route associated with the minimum cost
    ans = cost_dict[minimum_cost]       
    
    # Return route and cost
    return ans, minimum_cost

# n squared complexity 

## Brute Force Timing Tests

In [17]:
route = getRoute(test_df,1)
t1 = createPermutations(route)
t2 = removeUnwantedPermutations(route[0], t1)
t3 = calculateCostPerPermutation(t2, aircrafts["A330"].get_maxfuel(), airports)

In [18]:
%%timeit
route = getRoute(test_df,1)

559 µs ± 67.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [19]:
%%timeit
t1 = createPermutations(route)


7.98 µs ± 498 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [20]:
%%timeit
t2 = removeUnwantedPermutations(route[0], t1)


6.92 µs ± 242 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [21]:
%%timeit
t3 = calculateCostPerPermutation(t2, aircrafts["A330"].get_maxfuel(), airports)


1.12 ms ± 60.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Unit tests

In [22]:
class bruteF_unit_tests(unittest.TestCase):
       
    def test_create_permutations(self):
        airports = {'DUB':"dublin",'WEX': "Wexford"}
        self.assertTrue(createPermutations(airports) == [('DUB', 'WEX'), ('WEX', 'DUB')])
           
    def test_removeUnwantedPermutations(self):
        airport = 'DUB'
        permutations = [('DUB', 'WEX', 'HOM'), ('WEX', 'DUB', 'HOM')]
        self.assertTrue(removeUnwantedPermutations(airport, permutations) == [('DUB', 'WEX', 'HOM')])
 
    def test_calculateCostPerPermutation(self):
        route = getRoute(test_df,1)
        t1 = createPermutations(route)
        t2 = removeUnwantedPermutations(route[0], t1)
        self.assertTrue(calculateCostPerPermutation(t2,aircrafts["A330"].get_maxfuel(),airports) == (['SNN', 'ORK', 'CDG', 'SIN', 'MAN', 'SNN'], 19773.0925))
    
unittest.main(argv=[''], verbosity=2, exit=False)
#     def test_createPermutations(self):
        

test_create_aircraft (__main__.aircraft_unit_tests) ... ok
test_get_distance (__main__.aircraft_unit_tests) ... ok
test_get_fuel (__main__.aircraft_unit_tests) ... ok
test_get_fuel_needed (__main__.aircraft_unit_tests) ... ok
test_get_maxfuel (__main__.aircraft_unit_tests) ... ok
test_get_name (__main__.aircraft_unit_tests) ... ok
test_get_units (__main__.aircraft_unit_tests) ... ok
test_refuel (__main__.aircraft_unit_tests) ... ok
test_repr (__main__.aircraft_unit_tests) ... ok
test_calculateCostPerPermutation (__main__.bruteF_unit_tests) ... ok
test_create_permutations (__main__.bruteF_unit_tests) ... ok
test_removeUnwantedPermutations (__main__.bruteF_unit_tests) ... ok

----------------------------------------------------------------------
Ran 12 tests in 0.017s

OK


# Linked List

### The following is the code for a linked list data structure which we planned to use to store the airport permutations, to show we could create various data structures. However, in order for itertools to work, it requires a data structure with index based access to work, and therefore a Linked List was not suitable.  

#### However, it is ready for go on any missions it's needed on in the future.

#### A full list of methods and their descriptions can be found in the comments in the cell below.

#### Correctness: Passed all unit tests

#### Speed:

#### Linked list vs Python dictionary

#### Create = 
 - Linked list 308ns
 - Python Dict 38ns

#### Check if empty =
- Linked list 150ns
- Python Dict 96ns

#### Insert 11 elements =
- Linked list 8.96 µs
- Python Dict 703 ns

#### Efficiency:
 
- find head = constant
- find tail = constant
- insert at x = constant if on the node, otherwise n
- insert at head = constant
- insert at tail = constant as it's stored
- traverse = n
- size = constant, as it's stored
- search = constant if on the node, otherwise n
- empty = constant
- remove = constant if on the node, otherwise n

#### Security: Name mangling for "private" variable types. Getters and setters (encapsulation).

#### Robustness: Code breaking commands were considered and dealt with (see design). 

#### Clarity: Code is indented and commented, with multiple cells used to break up code.

#### Maintainability: Code commented and laid out in segments for easier searching and alterations. 

### Design

#### Scalability: The linked list can store as many elements as the system memory can hold, and can be used in other programs other than this one.

#### Robustness: An error message is returned if a remove operation is performed on an empty linked list.

In [116]:
# Linked List class
class LinkedList:
    """
    Included methods:  
              get_size:                Return size of the linked list
              is_empty:                Return true if empty, false if not
              insert_at_head:          Insert new node at the head
              insert_at_tail:          Insert new node at the tail
              insert_at_tank:          Insert new node at rank r
              remove_at_head:          Remove element at the head
              remove_at_rank:          Remove element at rank r
              traverse:                Move through the linked list
              search:                  Search for an element within the linked list
    """
    
    # Create node for linked list
    class _Node:
    
        def __init__(self, value):
            self.value = value
            self.next = None
    
    # Initialise linked list, with head and tail pointing to nothing, and size = 0
    def __init__(self):
        self.head_value = None
        self.tail_value = None
        self.size = 0
               
    # Return size of the linked list        
    def get_size(self):
        return self.size
    
    # Return true if the value of the head is null
    def is_empty(self):
        return self.head_value == None
    
    # Insert node at head
    def insert_at_head(self, value):
        new_node = self._Node(value)
        if self.is_empty():
            self.head_value = new_node
            self.tail_value = new_node
        else:
            new_node.next = self.head_value
            self.head_value = new_node
        self.size += 1
        
    # Insert node at tail
    def insert_at_tail(self, value):
        new_node = self._Node(value)
        self.tail_value.next = new_node
        self.tail_value = new_node
        self.size += 1
        
    # Insert node at rank r    
    def insert_at_rank(self, r, value):
        new_node = self._Node(value)
        r = self.search(r)
        new_node.next = r.next
        r.next = new_node
        self.size += 1
        
    # Remove node at head
    def remove_at_head(self):
        if self.is_empty():
            return("Can't remove from an empty linked list")
        old_head = self.head_value
        self.head_value = self.head_value.next
        old_head.next = None
        self.size -= 1
    
    # Remove node at rank
    def remove_at_rank(self, r):
        if self.is_empty():
            return("Can't remove from an empty linked list") 
        r = self.search(r)
        old_node = r.next
        r.next = old_node.next
        old_node.next = None
        self.size -= 1
        
    # Move through the list
    def traverse(self):
        pointer = self.head_value
        while pointer != None:
            pointer = pointer.next
        return pointer
    
    # Search for value within the linked list
    def search(self, r):
        pointer = self.head_value
        found = False
        while pointer != None and found == False:
            if pointer.value == r:
                found = True
            else:
                pointer = pointer.next
        return pointer
    
# find head = constant
# find tail = constant
# insert at x = constant if on the node, otherwise n
# insert at head = constant
# insert at tail = constant as it's stored
# traverse = n
# size = constant, as it's stored
# search = constant if on the node, otherwise n
# empty = constant
# remove = constant if on the node, otherwise n


## Unit testing for Linked List 

In [117]:
class LinkedListTesters(unittest.TestCase):
    
    # Test if new linkedlist has no value for it's head (and thus empty)
    def test_head(self):
        t = LinkedList()
        self.assertEqual(t.head_value, None)
    
    # Test removing from an empty linked list
    def test_empty_remove(self):
        t = LinkedList()
        self.assertEqual(t.remove_at_head(), "Can't remove from an empty linked list")
        
    # Test if size is correct after certain amount of values entered
    def test_get_size(self):
        t = LinkedList()
        for letter in ["A", "B", "C", "D"]:
            t.insert_at_head(letter)
        self.assertEqual(t.get_size(), 4)
    
    # Test if linked list is empty
    def test_is_empty(self):
        t = LinkedList()
        self.assertTrue(t.is_empty())
    
    # Test if linked list is empty after nodes added
    def test_insert_at_head_1(self):
        t = LinkedList()
        for letter in ["A", "B", "C", "D"]:
            t.insert_at_head(letter)
        self.assertFalse(t.is_empty())
        
    # Test that last node's value inserted at the head is the value of head 
    def test_insert_at_head_2(self):
        t = LinkedList()
        for letter in ["A", "B", "C", "D"]:
            t.insert_at_head(letter)
        self.assertEqual(t.head_value.value, "D")
        
    # Test that first node's value inserted at the head is the value of tail
    def test_insert_at_head_3(self):
        t = LinkedList()
        for letter in ["A", "B", "C", "D"]:
            t.insert_at_head(letter)
        self.assertEqual(t.tail_value.value, "A")
        
    # Test that the linked list is traversable, and when it reaches the end it returns null
    def test_traverse(self):
        t = LinkedList()
        for letter in ["A", "B", "C", "D"]:
            t.insert_at_head(letter)
        self.assertEqual(t.traverse(), None)    
        
    # Test that searching for a value after it has been entered returns that value
    def test_search_1(self):
        t = LinkedList()
        for letter in ["A", "B", "C", "D"]:
            t.insert_at_head(letter)
        self.assertEqual(t.search("C").value, "C")  
        
    # Test that searching for a value that has not been entered returns null
    def test_search_2(self):
        t = LinkedList()
        for letter in ["A", "B", "C", "D"]:
            t.insert_at_head(letter)
        self.assertEqual(t.search("Z"), None)  
        
    # Test that a node's value which has been inserted at the tail is the value of the tail
    def test_insert_at_tail(self):
        t = LinkedList()
        for letter in ["A", "B", "C", "D"]:
            t.insert_at_head(letter)
        t.insert_at_tail("Z")
        self.assertEqual(t.tail_value.value, "Z")
        
    # Test that inserting at a specific rank works
    def test_insert_at_rank(self):
        t = LinkedList()
        for letter in ["A", "B", "C", "D"]:
            t.insert_at_head(letter)
        t.insert_at_rank("B", "Z")
        self.assertEqual(t.search("B").next.value, "Z")
        
    # Test that removing at the head of the linked list works
    def test_remove_at_head(self):
        t = LinkedList()
        for letter in ["A", "B", "C", "D"]:
            t.insert_at_head(letter)
        t.remove_at_head()
        t.remove_at_head()
        self.assertEqual(t.head_value.value, "B")
        
    # Test that removing at rank works
    def test_remove_at_rank(self):
        t = LinkedList()
        for letter in ["A", "B", "C", "D"]:
            t.insert_at_head(letter)
        t.remove_at_rank("C")
        self.assertEqual(t.search("C").next.value, "A") 

unittest.main(argv=[''], verbosity=3, exit=False)

test_empty_remove (__main__.LinkedListTesters) ... ok
test_get_size (__main__.LinkedListTesters) ... ok
test_head (__main__.LinkedListTesters) ... ok
test_insert_at_head_1 (__main__.LinkedListTesters) ... ok
test_insert_at_head_2 (__main__.LinkedListTesters) ... ok
test_insert_at_head_3 (__main__.LinkedListTesters) ... ok
test_insert_at_rank (__main__.LinkedListTesters) ... ok
test_insert_at_tail (__main__.LinkedListTesters) ... ok
test_is_empty (__main__.LinkedListTesters) ... ok
test_remove_at_head (__main__.LinkedListTesters) ... ok
test_remove_at_rank (__main__.LinkedListTesters) ... ok
test_search_1 (__main__.LinkedListTesters) ... ok
test_search_2 (__main__.LinkedListTesters) ... ok
test_traverse (__main__.LinkedListTesters) ... ok
test_create_aircraft (__main__.aircraft_unit_tests) ... ok
test_get_distance (__main__.aircraft_unit_tests) ... ok
test_get_fuel (__main__.aircraft_unit_tests) ... ok
test_get_fuel_needed (__main__.aircraft_unit_tests) ... ok
test_get_maxfuel (__main__

### Run times

%%timeit shows more accurate timings, but doesn't store variables, and thus the code is also declare in non %%timeit cells

In [106]:
%%timeit
runTimeTestLL = LinkedList()

308 ns ± 14.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [107]:
%%timeit
pythonDict = {}

38.8 ns ± 4.15 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [108]:
runTimeTestLL = LinkedList()

In [109]:
pythonDict = {}

In [110]:
%%timeit
for i in ['A','B','C','D','E', 'F', 'G', 'H', 'I', 'J', 'k']:
    runTimeTestLL.insert_at_head(i)

8.96 µs ± 736 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [111]:
%%timeit
for i in ['A','B','C','D','E', 'F', 'G', 'H', 'I', 'J', 'k']:
    pythonDict[i] = 'placeholder'

703 ns ± 112 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [112]:
for i in ['A','B','C','D','E', 'F', 'G', 'H', 'I', 'J', 'k']:
    runTimeTestLL.insert_at_head(i)

In [113]:
for i in ['A','B','C','D','E', 'F', 'G', 'H', 'I', 'J', 'k']:
    pythonDict[i] = 'placeholder'

In [114]:
%%timeit
runTimeTestLL.is_empty()


150 ns ± 6.03 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [115]:
%%timeit
bool(pythonDict)


96 ns ± 5.63 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Summary

#### I’ve learned that proper planning in advance can make the implementation of a program easier, and deciding which algorithms and data structures to use determines how your final program will run, with certain trade-offs depending on what you used to complete the task. The importance of unit testing was also learned, as it helped guide the building of the program, with each unit test being a step towards the end goal.

#### I learned how valuable it was to choose the correct data structure and algorithms, and the importance of deciding what to keep and what to remove. We made a lot of extra functions which didn’t make the final cut because they would have added to the run time with no benefits.  

#### If I was to do the project again, I wouldn’t focus on getting as many data structures and algorithms packed in as I can just to show we know how to use them, and instead just use what I know I need now, and what I might need in the future.

#### Fom a technical perspective, I've learned about importing other people's work, permutations, and how to implement hash tables, graphs, and unit testing.
